In [30]:
import os
import time 
import numpy as np

import pinocchio as pin
from pinocchio.visualize.meshcat_visualizer import MeshcatVisualizer

Script_DIR = os.getcwd()
Urdf_path = os.path.join(Script_DIR,"New_Simscape_Assem", "urdf", "New_Simscape_Assem.urdf")
mesh_dir  = Script_DIR

model ,collision_model, visual_model = pin.buildModelsFromUrdf(Urdf_path, mesh_dir, pin.JointModelFreeFlyer())

print('Loaded model', model.name)

print(f"Model name  : {model.name}")
print(f"DOF (nv)    : {model.nv}")
print(f"Joints (nq) : {model.nq}")
print(f"Num joints  : {model.njoints}")
print(f"Num frames  : {model.nframes}")
print(f"Num bodies  : {model.nbodies}")
print(f"Num actuators: {model.nv}")

print("\n── Joints ──")
for i, name in enumerate(model.names):
    print(f"  [{i}] {name}")


Loaded model New_Simscape_Assem
Model name  : New_Simscape_Assem
DOF (nv)    : 26
Joints (nq) : 27
Num joints  : 22
Num frames  : 59
Num bodies  : 22
Num actuators: 26

── Joints ──
  [0] universe
  [1] root_joint
  [2] Hip_Frontal_BL
  [3] Hip_Sagittal_BL
  [4] Knee_BL
  [5] Ankle_BL
  [6] Knee_Fourbar_BL
  [7] Hip_Frontal_BR
  [8] Hip_Sagittal_BR
  [9] Knee_BR
  [10] Ankle_BR
  [11] Knee_FourBar_BR
  [12] Hip_Frontal_FL
  [13] Hip_Sagittal_FL
  [14] Knee_FL
  [15] Ankle_FL
  [16] Knee_FourBar_FL
  [17] Hip_Frontal_FR
  [18] Hip_Sagittal_FR
  [19] Knee_FR
  [20] Ankle_FR
  [21] Knee_FourBar_FR


In [31]:
import crocoddyl
state = crocoddyl.StateMultibody(model)
print("Crocoddyl state created with model")


Crocoddyl state created with model


In [38]:

viz = pin.visualize.MeshcatVisualizer(model, collision_model, visual_model)
viz.initViewer()
viz.loadViewerModel()


You can open the visualizer by visiting the following URL:
http://127.0.0.1:7005/static/


In [39]:

def sweep_joint(joint_name, amplitude=0.8, duration=4.0, dt=0.02):
    """Sinusoidally sweep a single joint for visualization testing."""
    joint_id = model.getJointId(joint_name)
    if joint_id >= model.njoints:
        print(f"Joint '{joint_name}' not found.")
        return

    idx_q = model.joints[joint_id].idx_q
    q = q0.copy()

    print(f"\nSweeping joint '{joint_name}' (idx_q={idx_q}) ...")
    t = 0.0
    while t < duration:
        q[idx_q] = amplitude * np.sin(2 * np.pi * t / duration)
        viz.display(q)
        time.sleep(dt)
        t += dt

q0 = pin.neutral(model)
sweep_joint("Knee_BL")



Sweeping joint 'Knee_BL' (idx_q=9) ...


In [37]:
def sweep_joint(joint_name, amplitude=0.8, duration=4.0, dt=0.02):
    joint_id = model.getJointId(joint_name)
    if joint_id >= model.njoints:
        print(f"Joint '{joint_name}' not found.")
        return

    idx_q = model.joints[joint_id].idx_q
    q = q0.copy()

    # Define mimic relationships: {mimic_joint: (parent_joint, multiplier, offset)}
    mimic_map = {
        "Ankle_FR":      ("Knee_FR",   1, 0),
        "Knee_FourBar_FR": ("Knee_FR", -1, 0),
        "Ankle_FL":      ("Knee_FL",   1, 0),
        "Knee_FourBar_FL": ("Knee_FL", -1, 0),
        "Ankle_BR":      ("Knee_BR",   1, 0),
        "Knee_FourBar_BR": ("Knee_BR", -1, 0),
        "Ankle_BL":      ("Knee_BL",   1, 0),
        "Knee_Fourbar_BL": ("Knee_BL", -1, 0),
    }

    print(f"\nSweeping joint '{joint_name}' (idx_q={idx_q}) ...")
    t = 0.0
    while t < duration:
        q[idx_q] = amplitude * np.sin(2 * np.pi * t / duration)

        # Enforce mimic joints
        for mimic_name, (parent_name, mult, offset) in mimic_map.items():
            parent_idx = model.joints[model.getJointId(parent_name)].idx_q
            mimic_idx = model.joints[model.getJointId(mimic_name)].idx_q
            q[mimic_idx] = mult * q[parent_idx] + offset

        viz.display(q)
        time.sleep(dt)
        t += dt

q0 = pin.neutral(model)
sweep_joint("Knee_BL")


Sweeping joint 'Knee_BL' (idx_q=9) ...


In [11]:
data = model.createData()

frame_name ="Foot_FR"
frame_id = model.getFrameId(frame_name)

q = pin.neutral(model)


pin.crba(model,data,q)
data.M =np.triu(data.M) +np.triu(data.M,1).T

print("Free flying coordinates: \n", q[:7])

print("Mass matrix :\n", data.M)
print(np.size(data.M))

Free flying coordinates: 
 [0. 0. 0. 0. 0. 0. 1.]
Mass matrix :
 [[ 5.43978520e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   3.40574317e-01  1.69167605e-05  1.53707638e-07  2.60133670e-02
   2.49130440e-02 -1.02391011e-04 -4.91386320e-03  7.77514805e-08
  -3.51705547e-02 -2.84098431e-02  1.12306512e-04  4.91379059e-03
   1.59223758e-07  2.03093907e-02  2.40050412e-02 -2.79758370e-04
  -4.91304482e-03  8.31545059e-08 -3.66872262e-02 -2.85791509e-02
   2.87494676e-04  4.91312185e-03]
 [ 0.00000000e+00  5.43978520e+00  0.00000000e+00 -3.40574317e-01
   0.00000000e+00  2.14810209e-01 -2.54629802e-02  2.48379804e-04
   2.95864841e-05 -2.83868682e-05 -2.22661264e-07 -3.54280336e-02
   4.43581415e-04  5.25896036e-05 -5.07439979e-05 -3.46898233e-07
  -2.70084454e-02 -1.07856382e-02  2.22011199e-05 -2.53911526e-05
   5.79866724e-07 -3.69338450e-02  4.53819241e-04  4.58864249e-05
  -5.29782880e-05  1.28936741e-06]
 [ 0.00000000e+00  0.00000000e+00  5.43978520e+00 -1.69167605e-05
  -2.14

In [40]:
# Joints that actually affect Foot_FR
leg_joints = ["Hip_Frontal_FR", "Hip_Sagittal_FR", "Knee_FR"]
leg_idxs_v = [model.joints[model.getJointId(j)].idx_v for j in leg_joints]

n_leg = len(leg_idxs_v)
S_leg = np.zeros((model.nv, n_leg))
for col, row in enumerate(leg_idxs_v):
    S_leg[row, col] = 1.0

# ── IK (position only, 3D) ────────────────────────────────────────
frame_id = model.getFrameId("Foot_FR")

q = pin.neutral(model)
enforce_mimic(q)

pin.forwardKinematics(model, data, q)
pin.updateFramePlacements(model, data)

target_pos = data.oMf[frame_id].translation.copy()
target_pos += np.array([0.03, 0.0, -0.05])

eps = 1e-4
dt = 1.0       # can be more aggressive with 3 DOFs
damp = 1e-6
max_iter = 1000

for i in range(max_iter):
    pin.forwardKinematics(model, data, q)
    pin.updateFramePlacements(model, data)

    # 3D position error only
    err = target_pos - data.oMf[frame_id].translation

    if np.linalg.norm(err) < eps:
        print(f"Converged in {i} iterations!")
        break

    # Position rows only (top 3 rows of Jacobian), in WORLD frame
    J_full = pin.computeFrameJacobian(model, data, q, frame_id,
                                       pin.ReferenceFrame.LOCAL_WORLD_ALIGNED)
    J = J_full[:3, :] @ S_leg  # 3 x 3

    # Damped solve
    dq_leg = np.linalg.solve(J.T @ J + damp * np.eye(n_leg), J.T @ err)

    # Map back to full q
    dq_full = S_leg @ dq_leg
    q = pin.integrate(model, q, dq_full * dt)
    enforce_mimic(q)

else:
    print(f"Did not converge. Final error: {np.linalg.norm(err):.6f}")

print("Target:  ", target_pos)
print("Achieved:", data.oMf[frame_id].translation)
viz.display(q)

Converged in 3 iterations!
Target:   [-0.00492305  0.08453364 -0.37366449]
Achieved: [-0.00492319  0.08453364 -0.37366437]
